In [ ]:
import pandas as pd
import pickle
from tqdm import tqdm
from sklearn.metrics import classification_report
import numpy as np

Mounted at /content/drive


In [ ]:
frag_train = pd.read_csv('../data/all_train_aligned.csv')
frag_dev = pd.read_csv('../data/all_dev_aligned.csv')
frag_test = pd.read_csv('../data/all_test_aligned.csv')


frag_train = frag_train[frag_train.apply(lambda x: type(x['0']) == str, axis = 1)]
frag_dev = frag_dev[frag_dev.apply(lambda x: type(x['0']) == str, axis = 1)]
frag_test = frag_test[frag_test.apply(lambda x: type(x['0']) == str, axis = 1)]

In [ ]:
with open('../../models/FREQ/freq_threshold_db.pkl', 'rb') as f:
  words_and_levels = pickle.load(f)

In [ ]:
def get_rl(token, model, oov_level = 0):
    try:
        return model[token]
    except:
        return oov_level

def levels_inference_pipeline(fragment, model, v = False):
  tokens = [t.split('#')[0] for t in fragment.split(' ')]
  levels = [get_rl(token, model, 0) for token in tokens]
  levels = [a if a > 3 else 3 for a in levels]
  if v:
    print(tokens)
    print(levels)

  return levels

In [1]:
def get_gt_levels(fragment):
  return [int(t.split('#')[1]) for t in fragment.split(' ')]

In [ ]:
gt_levels = [get_gt_levels(f) for f in frag_test['0']]

In [ ]:
predicted_levels_freq = [levels_inference_pipeline(a, words_and_levels) for a in frag_test['0']]

In [ ]:
preds = np.concatenate(predicted_levels_freq)
gt = np.concatenate(gt_levels)

In [ ]:
print(classification_report(gt, preds))

              precision    recall  f1-score   support

           3       0.95      0.82      0.88     23851
           4       0.18      0.28      0.22      2032
           5       0.20      0.60      0.30      1040

    accuracy                           0.77     26923
   macro avg       0.44      0.57      0.47     26923
weighted avg       0.86      0.77      0.81     26923

